<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорты" data-toc-modified-id="Импорты-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разбиение-на-выборки" data-toc-modified-id="Разбиение-на-выборки-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Разбиение на выборки</a></span></li><li><span><a href="#Получение-TF-IDF-для-корпуса-текста" data-toc-modified-id="Получение-TF-IDF-для-корпуса-текста-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Получение TF-IDF для корпуса текста</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Boosting" data-toc-modified-id="Boosting-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Boosting</a></span></li><li><span><a href="#Вert" data-toc-modified-id="Вert-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Вert</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация тональности комментариев


Обучим модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Значение метрики качества *F1* должно быть не меньше 0.75.** 

**Основные шаги по выполнению проекта**
1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Вывожы.

Для выполнения проекта применять *BERT* необязательно, но можно попробовать.

**Описание данных**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорты

In [1]:
import pandas as pd

import numpy as np
import re

# from pymystem3 import Mystem тормозит сильно
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import set_config
set_config(display="diagram")

from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 12345

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Danil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Подготовка

In [2]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except FileNotFoundError:
    df = pd.read_csv('datasets/toxic_comments.csv')
    
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [3]:
# отчистим сообщения
# сначала заменим все, что не буквы пробелами
# затем разобьем все на слова и объединим через пробел (чтобы убрать лишние пробелы)
df['text'] = df['text'].apply(lambda x: ' '.join(re.sub(r'[^a-zA-Z]', ' ', x).split()))

### Лемматизация 

Используемый мной ранее pymystem3 в данном случае не поможет, так как он работает только с руским текстом.

Лемматизатор нужно применять к словам, а не ко всему тексту сразу.
Использовал list comprehension для лемматизации, чтобы пройтись по каждому слову.
    
Думаю вместо очистки можно было воспользоваться nltk.word_tokenize

In [4]:
%%time
# лемматизируем

lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(
    lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.split()]))

df.head()

Wall time: 1min 19s


,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He match this background colour I m seem...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestion on impro...,0
4,You sir are my hero Any chance you remember wh...,0


## Обучение

### Разбиение на выборки

Приводить тексты к юникоду не имеет смысла, так как они все на английском. Это может привести к падению ядра из-за увеличения объема занимаемой памяти.

In [5]:
corpus = df['text']
target = df['toxic']

In [6]:
corpus_train, corpus_test, target_train, target_test = train_test_split(
    corpus, target, test_size=0.2, stratify=target, random_state=RANDOM_STATE)

print(corpus_train.shape)
print(corpus_test.shape)
print(target_train.value_counts(normalize=True))
print(target_test.value_counts(normalize=True))

(127656,)
(31915,)
0    0.89832
1    0.10168
Name: toxic, dtype: float64
0    0.898324
1    0.101676
Name: toxic, dtype: float64


### Получение TF-IDF для корпуса текста

Скачаем стоп-слова и обучим векторизатор только на тренировочной части выборки.

In [7]:
# получение стоп-слов 
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
%%time
tf_idf = TfidfVectorizer(stop_words=stop_words)

# обучаем только на тренировочной выборке
tf_idf_train = tf_idf.fit_transform(corpus_train)
tf_idf_test = tf_idf.transform(corpus_test)

Wall time: 12.8 s


### Логистическая регрессия

Внутри кросс-валидации происходит разбиение выборки на треин и валидацию. Однако, в таком случае векторизатор обучен на всей выборке, а это не совсем корректно.  
Для избежания такого эффекта можно использовать пйплайн.

In [9]:
# создадим функцию для вывода основных метрик качества класификации
def classification_metrics(target, predictions):
    print(f'Accuracy:  {accuracy_score(target, predictions):.2f}')
    print(f'Precision: {precision_score(target, predictions):.2f}')
    print(f'Recall:    {recall_score(target, predictions):.2f}')
    print(f'F1:        {f1_score(target, predictions):.2f}')
    print(f'ROC AUC:   {roc_auc_score(target, predictions):.2f}')
    print('_'*20)

In [10]:
%%time
model_lr = LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE).fit(tf_idf_train, target_train)

Wall time: 6.52 s


In [11]:
# создадим пайплайн для возможности корректной кросс-валидации
pipe_lr = Pipeline(
    [('tf_idf', TfidfVectorizer(stop_words=stop_words)),
     ('model', LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE))])
pipe_lr

Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 LogisticRegression(class_weight='balanced',
                                    random_state=12345))])

In [12]:
%%time
# произведем кросс-валидацию по метрике F1
cross_val_score(pipe_lr, corpus_train, target_train, cv=5, scoring='f1').mean()

Wall time: 1min 17s


0.7452011405904526

In [13]:
cross_val_score(model_lr, tf_idf_train, target_train, cv=5, scoring='f1').mean()

0.7479519895467238

In [14]:
classification_metrics(target_test, model_lr.predict(tf_idf_test))

Accuracy:  0.94
Precision: 0.68
Recall:    0.85
F1:        0.76
ROC AUC:   0.90
____________________


Пока не дотягиваем до требуемого минимального значения метрики F1 = 0.75 

Изменения:
1) В предыдущей редакции я никак не учитывал дисбаланс классов. Сейчас добавил праметр class_weight и качество логистической регрессии возрасло до целевого значения F1.  
2) Добавил pipeline для кросс-валидации. Ради интереса сравинил метрики - есть различия в тысячных.


### Решающее дерево

In [15]:
%%time
# model_dtc = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced').fit(tf_idf_train, target_train)
# cross_val_score(model_dtc, tf_idf_train, target_train, cv=5, scoring='f1').mean()

Wall time: 0 ns


In [16]:
%%time
# model_rfc = RandomForestClassifier(random_state=RANDOM_STATE).fit(tf_idf_train, target_train)

Wall time: 0 ns


### Boosting

In [17]:
%%time
model_lgbm = LGBMClassifier(
    learning_rate=0.1,
    n_estimators=100,
    class_weight='balanced',
    random_state=RANDOM_STATE)
model_lgbm.fit(tf_idf_train, target_train, verbose=10)

Wall time: 48.8 s


LGBMClassifier(class_weight='balanced', random_state=12345)

In [18]:
pipe_lgbm = Pipeline([
    ('tf_idf', TfidfVectorizer(stop_words=stop_words)),
    ('model', LGBMClassifier(
        learning_rate=0.1,
        n_estimators=100,
        class_weight='balanced',
        random_state=RANDOM_STATE))
])

pipe_lgbm

Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 LGBMClassifier(class_weight='balanced', random_state=12345))])

In [19]:
%%time
cross_val_score(pipe_lgbm, corpus_train, target_train, cv=3, scoring='f1').mean()

Wall time: 2min 14s


0.7312466239766078

In [20]:
classification_metrics(target_test, model_lgbm.predict(tf_idf_test))

Accuracy:  0.94
Precision: 0.68
Recall:    0.80
F1:        0.74
ROC AUC:   0.88
____________________


Пока я не разобрался как добавить pipeline для кросс-валидации из библиотеки catboost.  
Поэтому я решил заменить catboost на LGBM.
Оставлю блоки ниже закомментированным.

In [21]:
# %%time
# model_cbс = CatBoostClassifier(
#     iterations=100, learning_rate=0.9, depth=4, loss_function='Logloss', 
#     auto_class_weights='Balanced', random_state=RANDOM_STATE)
# model_cbс.fit(tf_idf_train, target_train, verbose=10)

In [22]:
# %%time

# # кроссвалидация из библиоткеи CatBoost

# params = {'iterations': 100,
#           'loss_function': 'Logloss',
#          'learning_rate': 0.9,
#          'depth': 4}

# cv_data = cv(
#     params=params,
#     pool=Pool(tf_idf_train, label=target_train),
#     fold_count=3, 
#     shuffle=True, # Перемешаем наши данные
#     plot=True, # визуализатор
#     verbose=False)


Увеличении скорости обучения у catboost позволило добится серрьезного улучшение качества. При learning_rate = 0.1, F1 = 0,67; при learning_rate = 1, F1 = 0,77;

Что немного для меня странно - в предыдущех проектах использовал небольшие значения этого коэффициента (но там были задачи регрессии). Возможно это связано с небольшим количестовм итераций. Здесь все довольно дого считается, поэтому ограничился 100 итерациями.

### Вert
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться (как в моем случае).

BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings) 
```

## Выводы

Целевые показатели качества по метрике F1 удалось достичь c помощью LightGBM.  
Случайный лес я в итоге не использовал - он всегда считается у меня достаточно долго.

Гиперпараметры пока перебирал только вручную. Потенциально для целей подбора можно использовать RandomizedSearch или OptunaSearch